In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os
import time

In [2]:
sulmatre = 'https://sulmatre.com.br/produtos/'
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [3]:
query = pd.read_csv('produtos_bot_yury_not_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [4]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [5]:
def findOEM(text):
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) >= 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [6]:
def sulmatreSearch(nav, cod, index):
    nav.get(sulmatre)
    elemento = nav.find_element(By.XPATH, '//*[@id="page"]/div[1]/div/section[1]/div/div/div/div/div/div/div/div/div/form/div/div')
    elemento = elemento.find_element(By.TAG_NAME, 'input')
    elemento.send_keys(cod)
    elemento.send_keys(Keys.ENTER)
    try:
        nome = nav.find_element(By.CLASS_NAME, 'product_title').text
        nome = nome.replace('|', '-')
        elementos = nav.find_elements(By.CLASS_NAME, 'woocommerce-product-gallery__image')
        src = []
        for elemento in elementos:
            img = elemento.find_element(By.TAG_NAME, 'img').get_attribute('src')
            src.append(img)
        keys = []
        values = []
        keys.append('Nome: ')
        values.append(nome)
        elementos = nav.find_elements(By.CLASS_NAME, 'woocommerce-product-attributes-item')
        for elemento in elementos:
            keys.append(elemento.find_element(By.CLASS_NAME, 'woocommerce-product-attributes-item__label').text)
            value = elemento.find_element(By.CLASS_NAME, 'woocommerce-product-attributes-item__value')
            values.append(value.find_element(By.TAG_NAME, 'p').text)
        keys.append('Imagens: ')
        values.append(src)
        dict_infos = dict(zip(keys, values))
        dict_infos = str(dict_infos)
        dict_infos = dict_infos.replace("'", '"')
        with open(f'{cwd}/Sulmatre/prods.json', 'a') as teste:
            teste.write(dict_infos)
    except Exception as e:
        notFound.loc[len(notFound)] = [index, 'Não encontrado no site']
        return
        pass

In [7]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)

In [8]:
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findOEM(cod)
    for item in cods:
        sulmatreSearch(nav, item, ID)

In [9]:
# sulmatreSearch(nav, '87617623', '1')

In [10]:
notFound = notFound.drop_duplicates()
notFound.to_excel(f'{cwd}/Sulmatre/notFound.xlsx')